In [12]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from newrow import newrow_cash, newrow_etf_stock
from utils import broker_fee, get_date, round_half_up, get_asset_value
from fetch_data import fetch_exchange_rate
import requests
from secret import OXCR_KEY
from datetime import date, timedelta, datetime
import json
import yfinance as yf


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:

# Crea un oggetto Ticker per la coppia di valute EUR/USD
eur_usd = yf.Ticker("USDEUR=X")

# Ottieni le informazioni più recenti
current_info = eur_usd.info

# Estrai il prezzo di acquisto (bid) e di vendita (ask) più recenti
current_rate_bid = current_info.get('bid')
current_rate_ask = current_info.get('ask')

print(f"Tasso di cambio EUR/USD attuale (Bid): {current_rate_bid}")
print(f"Tasso di cambio EUR/USD attuale (Ask): {current_rate_ask}")



Tasso di cambio EUR/USD attuale (Bid): 0.8592
Tasso di cambio EUR/USD attuale (Ask): 0.8597


In [ ]:
# Un altro modo per ottenere il prezzo di chiusura più recente
# Scarica i dati degli ultimi due giorni con un intervallo di un minuto
latest_data = yf.download("USDEUR=X", period="2d", interval="1m", progress=False)

# Ottieni l'ultimo prezzo di chiusura
latest_price = latest_data['Close'].iloc[-1]
print(f"Ultimo prezzo di chiusura EUR/USD: {latest_price}")

Ultimo prezzo di chiusura EUR/USD: Ticker
USDEUR=X    0.8591
Name: 2025-10-15 15:28:00+00:00, dtype: float64
0.8590999841690063


In [37]:
# Un altro modo per ottenere il prezzo di chiusura più recente
# Scarica i dati degli ultimi due giorni con un intervallo di un minuto
latest_data = yf.download("USDEUR=X",start="2025-10-15", end="2025-10-16", progress=False)

# Ottieni l'ultimo prezzo di chiusura
latest_price = latest_data['Close'].iloc[-1]
print(f"Ultimo prezzo di chiusura EUR/USD: {latest_price}")

Ultimo prezzo di chiusura EUR/USD: Ticker
USDEUR=X    0.8592
Name: 2025-10-15 00:00:00, dtype: float64
